# Data Cleaning Using Python

## Overview 

Netflix is one of the most popular media and video streaming platforms. They have over 8000 movies or tv shows available on their platform, as of mid-2021, they have over 200M Subscribers globally. This tabular dataset consists of listings of all the movies and tv shows available on Netflix, along with details such as - cast, directors, ratings, release year, duration, etc.

The main aim of this python script is to clean the data using Python libraries and then visualize it for useful insights using Tableau

## Data Cleaning

### Getting an overview of data for dealing with duplicate and null values

In [1]:
# importing required libraries
import pandas as pd
import numpy as np

In [2]:
# reading in the data into a pandas dataframe
data = pd.read_csv('netflix_titles.csv')

In [3]:
# a broad summary of the data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


As per above information, there are some columns with null values, which will be dealt with later in the code

In [4]:
# examining a few rows of the dataframe
data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


show_id is an id for every record in the dataset, so it should be unique.
The following code checks for duplicate enteries in this column

In [5]:
# checking duplicate records in show_id column
sum(data.duplicated(subset='show_id',keep=False)) 

0

There are no duplicate values in the column, suggesting no problem with data validation so far.
Similar process is repeated below for title column, because no two movies or TV shows shall have the same name. There are no duplicates in this column as well

In [6]:
sum(data.duplicated(subset='title',keep=False)) # no duplicates for title column, and it should be

0

In [7]:
# checking for number of null values in every column
data.isnull().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

The number of null values in director, cast and country column are not negliegible as compared to the number of records in the datasets, making it impossible to directly drop the rows containing null values for these columns. Thus, the null values are treated accordingly in the below sections of code

### Cleaning null values in director column 

Filling null values in director name is something that cannot be achieved by a mathematical logic. Like replacing null values with min,max,mean etc of a numerical column.

One of the ways to deal with null director names is to explore relationship between cast and director.
One can reasonably assume that if exactly same director,cast pair appears 3 or more times in the dataset, then in cases where director is null but the cast column is not null and contains the same cast as that in the pair, then it can be assumed that the director in that case is also the one in the pair 

So, in the below code, the director and cast columns are combined and then their repetition frequency is analyzed to further clean null values in the director column.

In [8]:
# combining director and cast columns into dcast column, with --- between the two column values for clear separation
data['dcast'] = data['director'] + '---' + data['cast']

Below, the values in the dcast column which appear more than 3 times in the dataset are stored in variable found directors. As these are the directors we will be able to replace if they are null but have the cast column as non-null and hold same value as given after ---

In [9]:
# storing the relevant values in variable found_directors
found_directors = list(data['dcast'].value_counts()[data['dcast'].value_counts()>=3].index)

In [10]:
found_directors

['Rajiv Chilaka---Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jigna Bhardwaj, Rajesh Kava, Mousam, Swapnil',
 'Rathindran R Prasad---Aishwarya Rajesh, Vidhu, Surya Ganapathy, Madhuri, Pavel Navageethan, Avantika Vandanapu',
 'S.S. Rajamouli---Prabhas, Rana Daggubati, Anushka Shetty, Tamannaah Bhatia, Sathyaraj, Nassar, Ramya Krishnan, Sudeep',
 'B. V. Nandini Reddy---Samantha Ruth Prabhu, Lakshmi, Rajendraprasad, Naga Shourya, Rao Ramesh, Teja Sajja, Pragathi, Jagapathi Babu, Aishwarya, Urvashi',
 'Louis C.K.---Louis C.K.',
 'Alastair Fothergill---David Attenborough',
 'Edward Cotterill---Rachael Stirling',
 'Stan Lathan---Dave Chappelle',
 'Walter C. Miller---Sam Kinison',
 'S.S. Rajamouli---Prabhas, Rana Daggubati, Anushka Shetty, Tamannaah Bhatia, Sathyaraj, Ramya Krishnan, Nassar, Subbaraju',
 'Ashwin Saravanan---Taapsee Pannu, Vinodhini, Parvathi T, Ramya Subramanian, Sanchana Natarajan, Anish Kuruvilla, David Solomon Raja',
 'Jay Karas---Bill Burr']

As can be seen above, there are 12 director---cast pairs which have been repeated 3 or more times.
So, if there is any record where director column is null, but the cast column is non-null and belongs to one of the casts above, then that null director will be replaced by the director value from above list

For example, if a record contains cast as 'Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jigna Bhardwaj, Rajesh Kava, Mousam, Swapnil', and director is null, then that director will be Rajiv Chilaka. Because Rajiv Chilaka has done 3 or more movies with the exact same cast.

In [11]:
# converting the above column in to two different values in a dictionary
direcast = dict()
for i in found_directors:
    director,cast = i.split('---')
    direcast[director]=cast

In [12]:
direcast

{'Rajiv Chilaka': 'Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jigna Bhardwaj, Rajesh Kava, Mousam, Swapnil',
 'Rathindran R Prasad': 'Aishwarya Rajesh, Vidhu, Surya Ganapathy, Madhuri, Pavel Navageethan, Avantika Vandanapu',
 'S.S. Rajamouli': 'Prabhas, Rana Daggubati, Anushka Shetty, Tamannaah Bhatia, Sathyaraj, Ramya Krishnan, Nassar, Subbaraju',
 'B. V. Nandini Reddy': 'Samantha Ruth Prabhu, Lakshmi, Rajendraprasad, Naga Shourya, Rao Ramesh, Teja Sajja, Pragathi, Jagapathi Babu, Aishwarya, Urvashi',
 'Louis C.K.': 'Louis C.K.',
 'Alastair Fothergill': 'David Attenborough',
 'Edward Cotterill': 'Rachael Stirling',
 'Stan Lathan': 'Dave Chappelle',
 'Walter C. Miller': 'Sam Kinison',
 'Ashwin Saravanan': 'Taapsee Pannu, Vinodhini, Parvathi T, Ramya Subramanian, Sanchana Natarajan, Anish Kuruvilla, David Solomon Raja',
 'Jay Karas': 'Bill Burr'}

The conversion to dictionary separates the values in to director name and cast. It is required because these two columns were combined just to see their joint frequency. Separating them and storing as a dictionary will allow for better lookup and cleaning as done in the next code snippet.

Below, the data records with null director and cast equal to the cast value of ith record in the direcast dictionary is taken and then assigned with the director value from the same ith record. It is achieved using .loc function in pandas.
direcast.items())[i][1] : .items basically makes a list of tuples of the director,cast values. [i] gets the records in serial order and then [1] gets the cast, as the tuple is of the form (director, cast). Then if both conditions, director null and cast equal to cast in direcast are met, then the director value from direcast is used to fill the null value in director column of the data.
The same procedure is followed for every director,cast pair in the direcast dictionary, as these are the only values which can be logically used to replace null director values.

In [13]:
for i in range(len(direcast)):
    data.loc[(data['director'].isna()) & (data['cast'] == list(direcast.items())[i][1]),'director'] = list(direcast.items())[i][0]

In [14]:
# checking again for the null values
data.isnull().sum()

show_id            0
type               0
title              0
director        2616
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dcast           3107
dtype: int64

Using the above logic, 18 null director values out of 2634 have been replaced. It is quite less as compared to the overall values, but there is no other logical way to move forward in this case.
The remaining null values are assigned 'Not Given' value in the below code'

In [15]:
data.loc[data['director'].isna(),'director'] = 'Not Given'
# Assign Not Given to all other directors
# There is no other way to fill in null values, apart from actually searching for movie directors online and then manually filling them one by one

### Cleaning null values in country column 

For cleaning null values in country column, the logic is almost similar to the one used for cleaning null directors. The country can be estimated from the director and movie because they are related to each other. So, the below code uses director information, wherever available, to replace null country values.

In [16]:
# making a dictionary to store director and it's country as a pair
direc_cntry = dict(list(zip(data['director'],data['country'])))
direc_cntry

{'Kirsten Johnson': 'United States',
 'Not Given': nan,
 'Julien Leclercq': 'France, Belgium',
 'Mike Flanagan': 'United States',
 'Robert Cullen, José Luis Ucha': nan,
 'Haile Gerima': 'United States, Ghana, Burkina Faso, United Kingdom, Germany, Ethiopia',
 'Andy Devonshire': 'United Kingdom',
 'Theodore Melfi': 'United States',
 'Kongkiat Komesiri': nan,
 'Christian Schwochow': 'Germany, Czech Republic',
 'Bruno Garotti': 'Brazil',
 'Pedro de Echave García, Pablo Azorín Williams': nan,
 'Adam Salky': nan,
 'Olivier Megaton': 'United States',
 'K.S. Ravikumar': nan,
 'Alex Woo, Stanley Moore': 'United States',
 'S. Shankar': 'India',
 'Rajiv Menon': nan,
 'Dennis Dugan': 'United States',
 'Scott Stewart': 'United States',
 'Robert Luketic': 'United States',
 'Ashwiny Iyer Tiwari, Abhishek Chaubey, Saket Chaudhary': nan,
 'Daniel Sandu': nan,
 'Cédric Jimenez': nan,
 'George Nolfi': 'China, Canada, United States',
 'Rajiv Chilaka': nan,
 'Steven Spielberg': 'United States, India',
 'J

In [17]:
# same as the director column, the country values from direc_cntry are used to fill in null country values
for i in range(len(direc_cntry)):
    data.loc[(data['country'].isna()) & (data['director'] == list(direc_cntry.items())[i][0]),'country'] = list(direc_cntry.items())[i][1]

In [18]:
# checking for the null values after cleaning
data.isna().sum()

show_id            0
type               0
title              0
director           0
cast             825
country          734
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dcast           3107
dtype: int64

Based on the above logic, 97 out of 831 null records in country have been cleaned. The rest of the columns are replaced by 'Not Given' in the below code line

In [19]:
data.loc[data['country'].isna(),'country'] = 'Not Given'

### Cleaning null values in date_added, rating and duration columns

Number of null values in date_added, rating and duration columns is very small as compared to total records, so no effort is done cleaning these values and the records containing null values for these columns are simply dropped

In [20]:
# dropping records
data.drop(data[data['date_added'].isna()].index,axis=0,inplace=True)

In [21]:
# dropping records
data.drop(data[data['rating'].isna()].index,axis=0,inplace=True)
data.drop(data[data['duration'].isna()].index,axis=0,inplace=True)

In [22]:
# description column will be not be needed for visualization as it contains explanatory text, so this column is also dropped
data.drop(['dcast','description'],axis=1,inplace=True)

In [23]:
# null values in cast column are also replaced with Not Given
data.loc[data['cast'].isna(),'cast'] = 'Not Given'

In [24]:
# final check for number of null values in every column
data.isna().sum()

show_id         0
type            0
title           0
director        0
cast            0
country         0
date_added      0
release_year    0
rating          0
duration        0
listed_in       0
dtype: int64

As seen above, all null values have been dealt with

### Checking the validity or inconsistency if any in date_added and release_year

It can be simply understood that date_added column should not contain any date value that comes before release_year. Because how can a movie be added on Netflix before it has even been released. So, the below code checks these inconsistencies and then deal with them appropriately

In [25]:
# the date_added column is converted to datetime data type for further analysis using date functions
import datetime as dt
data['date_added_date'] = pd.to_datetime(data['date_added'])

In [26]:
# check if there are any added_dates that come before release_year
sum(data['date_added_date'].dt.year < data['release_year'])

14

As it can be seen, there are 14 records where the show or movie was added to netflix before it was released. Below code shows those records

In [27]:
data.loc[(data['date_added_date'].dt.year < data['release_year']),['date_added_date','release_year']]

,date_added_date,release_year
1551,2020-12-14,2021
1696,2020-11-15,2021
2920,2020-02-13,2021
3168,2019-12-06,2020
3287,2019-11-13,2020
3369,2019-10-25,2020
3433,2019-10-11,2020
4844,2018-05-30,2019
4845,2018-05-29,2019
5394,2017-07-01,2018


So, for dealing with these records, it is reasonable to assume that there has been a mistake in filling out the release years for these movies. Because date_added is supposed to be something that comes from the datalog system of netflix and is not supposed to be wrong. So, it is reasonable to alter the release year according to data added column as done in below code

In [28]:
# release_year values are replaced with year values from their respective date_added values
data.loc[(data['date_added_date'].dt.year < data['release_year']),'release_year'] = data['date_added_date'].dt.year

In [29]:
# some of the records above are checked and it is confirmed that they have been accurately replaced
data.iloc[[1551,1696,2920,3168]]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,date_added_date
1551,s1552,TV Show,Hilda,Not Given,"Bella Ramsey, Ameerah Falzon-Ojo, Oliver Nelso...","United Kingdom, Canada, United States","December 14, 2020",2020,TV-Y7,2 Seasons,Kids' TV,2020-12-14
1696,s1697,TV Show,Polly Pocket,Not Given,"Emily Tennant, Shannon Chan-Kent, Kazumi Evans...","Canada, United States, Ireland","November 15, 2020",2020,TV-Y,2 Seasons,Kids' TV,2020-11-15
2920,s2921,TV Show,Love Is Blind,Not Given,"Nick Lachey, Vanessa Lachey",United States,"February 13, 2020",2020,TV-MA,1 Season,"Reality TV, Romantic TV Shows",2020-02-13
3168,s3169,TV Show,Fuller House,Not Given,"Candace Cameron Bure, Jodie Sweetin, Andrea Ba...",United States,"December 6, 2019",2019,TV-PG,5 Seasons,TV Comedies,2019-12-06


Netflix was launched in 1997. So if there is any record that contains an addition to Netflix before this year, is supposed to be wrong and shall be dealt with

In [111]:
# checking if there is any addition to Netflix before 1997
sum(data['date_added_date'].dt.year < 1997)
# turns out that there is None

0

### Taking an overview of values in some columns for any invalid or inconsistent values 

Columns like rating, duration and type are checked for their unique values. This is done to spot any invalid or inconsistent values, in case it exists. Based on below results, there does not seem to be any problem

In [33]:
# checking unique values for rating column
data['rating'].unique()

array(['PG-13', 'TV-MA', 'PG', 'TV-14', 'TV-PG', 'TV-Y', 'TV-Y7', 'R',
       'TV-G', 'G', 'NC-17', 'NR', 'TV-Y7-FV', 'UR'], dtype=object)

In [38]:
# checking unique values for duration column
data['duration'].unique()

array(['90 min', '2 Seasons', '1 Season', '91 min', '125 min',
       '9 Seasons', '104 min', '127 min', '4 Seasons', '67 min', '94 min',
       '5 Seasons', '161 min', '61 min', '166 min', '147 min', '103 min',
       '97 min', '106 min', '111 min', '3 Seasons', '110 min', '105 min',
       '96 min', '124 min', '116 min', '98 min', '23 min', '115 min',
       '122 min', '99 min', '88 min', '100 min', '6 Seasons', '102 min',
       '93 min', '95 min', '85 min', '83 min', '113 min', '13 min',
       '182 min', '48 min', '145 min', '87 min', '92 min', '80 min',
       '117 min', '128 min', '119 min', '143 min', '114 min', '118 min',
       '108 min', '63 min', '121 min', '142 min', '154 min', '120 min',
       '82 min', '109 min', '101 min', '86 min', '229 min', '76 min',
       '89 min', '156 min', '112 min', '107 min', '129 min', '135 min',
       '136 min', '165 min', '150 min', '133 min', '70 min', '84 min',
       '140 min', '78 min', '7 Seasons', '64 min', '59 min', '139 min',
    

In [91]:
# checking unique values for type column
data['type'].unique()

array(['Movie', 'TV Show'], dtype=object)

### Data Transformation of "listed_in" column for additional granularity

The listed_in column contains categories under which a particular movie or TV show is listed. In this column, various categories are just clubbed together using comma as a separator. It may be beneficial to have them separated out also, in case one wants to analyze each category separately.

In [98]:
# checking the maximum number of categories there are in any record for listed_in column
max(data['listed_in'].str.split(',').apply(lambda x: len(x)))

3

In [107]:
# splitting the listed_in column and storing the values in three nee separate columns
data['listed_in1'] = data['listed_in'].str.split(',',expand = True)[0]
data['listed_in2'] = data['listed_in'].str.split(',',expand = True)[1]
data['listed_in3'] = data['listed_in'].str.split(',',expand = True)[2]
# three columns used because the maximum number of categories that will be there after split is 3
# if after split the categories < 3, then the remaining columns will be filled with null value

In [108]:
# final inspection of data by viewing a few rows
data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,date_added_date,listed_in1,listed_in2,listed_in3
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Not Given,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,2021-09-25,Documentaries,None,None
1,s2,TV Show,Blood & Water,Not Given,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries",2021-09-24,International TV Shows,TV Dramas,TV Mysteries
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...","France, Belgium","September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",2021-09-24,Crime TV Shows,International TV Shows,TV Action & Adventure
3,s4,TV Show,Jailbirds New Orleans,Not Given,Not Given,Not Given,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV",2021-09-24,Docuseries,Reality TV,None
4,s5,TV Show,Kota Factory,Not Given,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",2021-09-24,International TV Shows,Romantic TV Shows,TV Comedies


# Storing cleaned data in to a new CSV file

In [113]:
data.to_csv('Netflix_Data.csv',sep=',',header=True,index=False)

This is the end of data wrangling in Python. Now the further visualization and analysis is done using Tableau